# Intro

blah blaj

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing
import math
from sklearn import svm


## Reading file from gzip

In [2]:
df = pd.read_csv('autos.csv.gz', sep=',', header=0, compression='gzip',encoding='cp1252')

## Cleaning data from outliers and dirty values

In [3]:
# Cleaning data
valid_models = df.loc[df['model'] != 'NaN']

#### Removing the duplicates
dedups = valid_models.drop_duplicates(['name','seller','offerType','price','abtest','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage','postalCode'])

#### Removing the outliers
no_outliers = dedups[(df.yearOfRegistration <=  2016) 
      & (df.yearOfRegistration >= 1950) 
      & (df.price >= 500) 
      & (df.price <= 50000) 
      & (df.powerPS >= 30) 
      & (df.powerPS <= 500) 
      & (df.kilometer >= 1000) 
      & (df.kilometer <= 200000)]

#rel_cols['price'].unique()
#rel_cols['yearOfRegistration'].unique()
#rel_cols['gearbox'].unique()
#rel_cols['powerPS'].unique()
#rel_cols['model'].unique()
#rel_cols['kilometer'].unique()
#rel_cols['monthOfRegistration'].unique()
#rel_cols['brand'].unique()
#rel_cols['notRepairedDamage'].unique()


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Selecting only relevant columns

In [4]:

# only relevant columns
rel_cols = no_outliers[['price'
                        ,'yearOfRegistration'
                        ,'gearbox'
                        ,'powerPS'
                        ,'model'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ,'brand'
                        ,'notRepairedDamage']]


In [5]:

le_gearbox = preprocessing.LabelEncoder()
le_notRepairedDamage = preprocessing.LabelEncoder()
le_model = preprocessing.LabelEncoder()
le_brand = preprocessing.LabelEncoder()

le_gearbox.fit(rel_cols['gearbox'])
tr = le_gearbox.transform(rel_cols['gearbox']) 
rel_cols.loc[:, 'gearbox_feat'] = pd.Series(tr, index=rel_cols.index)

le_notRepairedDamage.fit(rel_cols['notRepairedDamage'])
tr = le_notRepairedDamage.transform(rel_cols['notRepairedDamage']) 
rel_cols.loc[:, 'notRepairedDamage_feat'] = pd.Series(tr, index=rel_cols.index)

le_model.fit(rel_cols['model'])
tr = le_model.transform(rel_cols['model']) 
rel_cols.loc[:, 'model_feat'] = pd.Series(tr, index=rel_cols.index)

le_brand.fit(rel_cols['brand'])
tr = le_brand.transform(rel_cols['brand']) 
rel_cols.loc[:, 'brand_feat'] = pd.Series(tr, index=rel_cols.index)

autos = rel_cols[ ['price'
                        ,'yearOfRegistration'
                        ,'gearbox_feat'
                        ,'powerPS'
                        ,'model_feat'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ,'brand_feat'
                        ,'notRepairedDamage_feat']]

Y = autos['price']
X = autos.drop(['price'], axis=1)

print(len(Y))
print(len(X))

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]
/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(valu

289893
289893


## Linear Regression test without normalization

In [7]:
linear = linear_model.LinearRegression()
train_len = int(len(X)*.8)
X_train = X[:train_len]
Y_train = Y[:train_len]
linear.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
#print(X, Y)

X_test = X[train_len+1:]
Y_test = Y[train_len+1:]

Y_predict = linear.predict(X_test)
print(linear.score(X_test, Y_test) ) 

0.60166638059


## SVR without normalization

In [ ]:
svr_poly = svm.SVR(kernel='linear', C=1e2, degree=2, verbose=True)
svr_poly.fit(X,Y)


Y_predict = svr_poly.predict(X_test)
print(svr_poly.score(X_test, Y_test) ) 

[LibSVM]